In [1]:
import pandas as pd
import numpy as np
import zipfile
import sys, os

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## 载入并查看数据

首先，在 [Global Terrorism Database](https://www.kaggle.com/START-UMD/gtd)网页下载数据集，下面我们之间对其进行操作：

In [2]:
filename = '../data/globalterrorismdb_0718dist.csv_3.zip'

zf = zipfile.ZipFile(filename)

zf.extractall('./data/')    # 解压数据
zf.close()

gtd_file = os.listdir('data/')
gtd_file

['globalterrorismdb_0718dist.csv']

该文件是 `.csv` 格式，需要解决各种编码问题，有点不是那么友好：

In [3]:
%%time
df = pd.read_csv('data/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1', low_memory=False)  # 载入数据

Wall time: 5.96 s


载入该文件花费时间小于 $10$ 秒.

In [4]:
df.get_dtype_counts()  # 查看数据类型分布

float64    55
int64      22
object     58
dtype: int64

我们可以将其转换为 Excel (`.xlsx` 格式) ：

In [5]:
df.to_excel('data/globalterrorismdb_0718dist.xlsx')  # 保存为 excel 格式

In [6]:
%%time
df_ex = pd.read_excel('data/globalterrorismdb_0718dist.xlsx')

Wall time: 1min 37s


载入该 excel 文件花费 `1min 45s` 也不是我们想要的结果. 为了加快数据的存储和载入, 我们采用 HDF5 格式来保存文件：

In [7]:
df.to_hdf('data/globalterrorismdb_0718dist.h5', key='gtd_df')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['approxdate', 'resolution', 'country_txt', 'region_txt', 'provstate', 'city', 'location', 'summary', 'alternative_txt', 'attacktype1_txt', 'attacktype2_txt', 'attacktype3_txt', 'targtype1_txt', 'targsubtype1_txt', 'corp1', 'target1', 'natlty1_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2_txt', 'targtype3_txt', 'targsubtype3_txt', 'corp3', 'target3', 'natlty3_txt', 'gname', 'gsubname', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'motive', 'claimmode_txt', 'claimmode2_txt', 'claimmode3_txt', 'weaptype1_txt', 'weapsubtype1_txt', 'weaptype2_txt', 'weapsubtype2_txt', 'weaptype3_txt', 'weapsubtype3_txt', 'weaptype4_txt', 'weapsubtype4_txt', 'weapdetail', 'propextent_txt', 'propcomment', 'divert', 'kidhijcountry', 'ransom

In [8]:
%%time
se = pd.read_hdf('data/globalterrorismdb_0718dist.h5')

Wall time: 2.62 s


载入 HDF5 文件仅仅需要 $2.42$ 秒。但是，有个性能预警，为此需要对其进行处理：

### 查看数据类型的分布情况

In [10]:
# Display a summary of the data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB


GTD 数据集有 $181\,691$ 个样本，且其特征数目为 $135$。为了解决 `PerformanceWarning` 问题， 下面我们仅仅需要处理 `object` 数据类型数据。

In [11]:
txt_columns = df.select_dtypes(['object']).columns
df.loc[:,txt_columns] = df.loc[:,txt_columns].replace(np.nan, 'Unknown')

In [14]:
%%time
df.to_hdf('data/globalterrorismdb_0718dist.h5', key='gtd_df')

Wall time: 3.5 s


此时，便没有在报错！

In [15]:
%%time
gtd_df = pd.read_hdf('data/globalterrorismdb_0718dist.h5')  #  载入数据

Wall time: 1.78 s


**总结**：相比 Excel 和 CSV，HDF5 的载入和写入都很快速和方便！下面还需要对 GTD 数据进行一些必要的操作，详细内容见：[01-数据预处理](./01-数据预处理.ipynb).